In [702]:
import pandas as pd
import json

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [703]:
dataset = pd.read_csv('description_predictions.csv', index_col=0)
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [704]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

dataset_withoutNAN.reset_index(drop=True, inplace=True)

Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [705]:
for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description

In [706]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
0,home team and away team take on each other in ...,0
1,home team begin their title defence with a mat...,1
2,away team will look to make a flying start to ...,2
3,newcomers home team will be aiming to kick off...,2
4,after failing to impress in the 2020/21 serie ...,2


In [707]:
from nltk.corpus import stopwords
import nltk
stop_words = set(stopwords.words('english'))

# this function returns a list of tokenized and stemmed words of any text
def get_tokenized_list(doc_text):
    tokens = nltk.word_tokenize(doc_text)
    return tokens

# This function will performing stemming on tokenized words
def word_stemmer(token_list):
  ps = nltk.stem.PorterStemmer()
  stemmed = []
  for words in token_list:
    stemmed.append(ps.stem(words))
  return stemmed

In [708]:
# Function to remove stopwords from tokenized word list
def remove_stopwords(doc_text):
  cleaned_text = []
  for words in doc_text:
    if words not in stop_words:
      cleaned_text.append(words)
  return cleaned_text

In [709]:
cleaned_corpus = []
for doc in dataset_withoutNAN.description:
  tokens = get_tokenized_list(doc)
  doc_text = remove_stopwords(tokens)
  doc_text  = word_stemmer(doc_text)
  doc_text = ' '.join(doc_text)
  cleaned_corpus.append(doc_text)

In [710]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [711]:

vectorizer = TfidfVectorizer(ngram_range=(2,2)) #vectorizer sarà il nostro modello da allenare
text_counts = vectorizer.fit_transform(cleaned_corpus) #impara il vocabolario e crea Idf, poi ritorna la matrice document-term
df1 = pd.DataFrame(text_counts.toarray(), columns=vectorizer.get_feature_names())

In [712]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, shuffle=False)

In [713]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3).fit(X_train)


Nel seguente codice applico il k-means

In [714]:
import sklearn.metrics as mtr
import numpy as np 
import itertools

label_classes = {
    'N': 0,
    'V': 1,
    'P': 2
}
inverted_label_classes = {v: k for k, v in label_classes.items()}

classes = [0,1,2]

#trovo tutte le combinazioni da 3 classi (0 = 'N', 1 = 'V', 2 = 'P')
all_combinations = list(itertools.product(*[classes,classes,classes]))
combinations = []
#filtro le combinazioni con le 3 classi diverse: la funzione precedente mi dà anche AAA AAC AAB ecc. sono classi sporche, quindi le elimino
for single_combination in all_combinations:
    if single_combination[0]!=single_combination[1] and single_combination[0]!=single_combination[2] and single_combination[1]!=single_combination[2]:
        combinations.append([(0,single_combination[0]), (1,single_combination[1]), (2,single_combination[2])])

Quando faccio il k means i documenti simili vengono raggruppati tra di loro, ma non so effettivamente se è corretta l'etichetta che viene assegnata a ciascun cluster.
In poche parole, tutte le partite che vedono come vincitrice la squadra di home dovrebbero essere raggruppate sotto l'etichetta '1', ma può essere che l'etichetta sia 0 e quando si misura l'accuratezza ovviamente avrò un risultato basso. 
Per questo devo trovare tutte le combinazioni possibili di etichette: 
- Cluster1, Cluster2, Cluster3 --> 0, 1, 2
- Cluster1, Cluster2, Cluster3 --> 0, 2, 1
- Cluster1, Cluster2, Cluster3 --> 1, 0, 2
- Cluster1, Cluster2, Cluster3 --> 1, 2, 0
- Cluster1, Cluster2, Cluster3 --> 2, 0, 1
- Cluster1, Cluster2, Cluster3 --> 2, 1, 0
In questo modo poi vado a vedere quale di questi cluster ha l'accuratezza migliore e tengo.

Itero ciascuna combinazione e calcolo l'accuratezza di quella combinazione con y_test e prendo l'accuratezza maggiore. 
Successivamente applico la combinazione che dà maggiore accuratezza al k_means e la stampo

In [715]:
best_score = 0 #nessun kmeans darà 0, quindi se lo trovo maggiore ho trovato il miglior score locale
best_kmean = []

initial_labels = np.unique(y_test)
original_prediction = kmeans.predict(X_test)

for combination in combinations:
    prediction = original_prediction.copy()
    for convertion in combination:
        prediction[prediction == convertion[0]] = ord(inverted_label_classes[convertion[1]])        
    for letter, single_class in label_classes.items():
        prediction[prediction == ord(letter)] = single_class
    
    score = mtr.accuracy_score(y_test, prediction)
    if score > best_score:
        best_score = score
        best_kmean = combination #tengo la combinazione delle classi

In [716]:
if best_kmean:
    for convertion in best_kmean:
        original_prediction[original_prediction == convertion[0]] = ord(inverted_label_classes[convertion[1]])
    for letter, single_class in label_classes.items():
        original_prediction[original_prediction == ord(letter)] = single_class
    
score = mtr.accuracy_score(y_test, original_prediction)
score, original_prediction

(0.4342105263157895,
 array([0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 2, 1, 0, 2, 1, 1, 0,
        1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 1, 0], dtype=int32))